In [6]:
using LinearAlgebra
A = rand(100, 100)

100×100 Matrix{Float64}:
 0.622132   0.979937   0.878121   …  0.60163    0.829046    0.347172
 0.381711   0.186892   0.269061      0.271835   0.616504    0.979965
 0.640267   0.367854   0.440008      0.754427   0.309994    0.734078
 0.381581   0.0710208  0.104405      0.727562   0.779348    0.236516
 0.906111   0.0247212  0.110433      0.861573   0.791628    0.358224
 0.512759   0.830195   0.628176   …  0.241672   0.183572    0.956419
 0.25001    0.106591   0.470208      0.144881   0.778122    0.548307
 0.530193   0.372534   0.482159      0.880779   0.821807    0.686484
 0.602836   0.781622   0.375112      0.956534   0.351681    0.721121
 0.701773   0.860514   0.0959624     0.928031   0.958223    0.126019
 0.0379015  0.139954   0.445724   …  0.375712   0.931331    0.529284
 0.251595   0.747832   0.943856      0.922851   0.207883    0.979375
 0.872945   0.173492   0.396486      0.0101451  0.105432    0.0468076
 ⋮                                ⋱                         
 0.912878   0.02

In [70]:
"""evaluates inverse of A' = (A  - λB), where B is diagonal.
Uses _Sherman–Morrison_ formula for rank-one update. It is still O(N^3) by time, but without extra memory cost
Arguments:
- Ainv::Matrix{T} - original matrix inverse
- bdiag::Vector - diagonal of correction matrix B.
- lam -- λ value.

This function overwrites **Ainv**. Returns true if finished succesfully (i.e lam != bdiag[kk] for all kk)
"""
function inv_diagupdate!(Ainv::Matrix{T}, bdiag::Vector{T}, lam::T) where {T}
    N = length(bdiag)
    @assert N>0
    @inbounds @fastmath for kk=1:N, jj=1:N, ii=1:N
        koeff = (1 - lam*bdiag[kk])
        if abs.(koeff)<eps(T)
            #@warn "inv_diagupdate: lam=$lam==1/bdiag[$kk]"
            return false
        end
        Ainv[ii, jj] += Ainv[ii, kk]*Ainv[kk, jj]/koeff
    end
    return true
end

inv_diagupdate!

In [75]:
A = rand(100, 100)
B = rand(size(A,2))
A2 = A + diagm(B) 
@time A2inv = inv(A); 
@time inv_diagupdate!(A2inv, B, -1e0)
@time inv(A2);
A2inv

  0.000452 seconds (9 allocations: 129.203 KiB)
  0.001676 seconds
  0.000345 seconds (9 allocations: 129.203 KiB)


100×100 Matrix{Float64}:
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN

In [44]:
@code_warntype inv_diagupdate!(A2inv, B, 1e0)

MethodInstance for inv_diagupdate!(::Matrix{Float64}, ::Vector{Float64}, ::Float64)
  from inv_diagupdate!(Ainv::Matrix{T}, bdiag::Vector{T}, lam::T) where T @ Main In[42]:10
Static Parameters
  T = Float64
Arguments
  #self#::Core.Const(Main.inv_diagupdate!)
  Ainv::Matrix{Float64}
  bdiag::Vector{Float64}
  lam::Float64
Locals
  @_5::Union{Nothing, Tuple{Int64, Int64}}
  N::Int64
  @_7::Union{Nothing, Tuple{Int64, Int64}}
  jj@_8::Int64
  @_9::Union{Nothing, Tuple{Int64, Int64}}
  ii@_10::Int64
  jj@_11::Int64
  kk::Int64
  koeff::Float64
  ii@_14::Int64
Body::Bool
1 ──       (N = Main.length(bdiag))
│    %2  = N::Int64
│    %3  = (1:%2)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│          (@_5 = Base.iterate(%3))
│    %5  = @_5::Union{Nothing, Tuple{Int64, Int64}}
│    %6  = (%5 === nothing)::Bool
│    %7  = Base.not_int(%6)::Bool
└───       goto #12 if not %7
2 ┄─ %9  = @_5::Tuple{Int64, Int64}
│          (jj@_8 = Core.getfield(%9, 1))
│    %11 = Core.getfiel